In [ ]:
import os
import numpy as np
import xarray as xr
import pandas as pd

In [ ]:
# Define the folder path where your NetCDF files are located
folder_path = 'E:/SMOS20192022/TB_2022_D'

In [ ]:
# Define the coordinates for which you need data
coordinates = [
(41.940084,	-73.49722),
(41.94195,	-73.4951),
(41.95597,	-73.57178),
(41.924,	-73.61745),
(41.9501,	-73.5794),
(41.9501,	-73.61745),
(41.8385,	-73.61913),
(41.8278,	-73.50703),
(41.79884,	-73.73929),
(41.80126,	-73.73325),
(41.802,	-73.6272),
(41.96187,	-73.71193),
(41.94754,	-73.74296),
(41.90908,	-73.74947),
(41.89083,	-73.78043),
(41.8766, -73.76864),
(41.81892,	-73.66515),
(41.86077,	-73.64758),
(41.71343,	-73.63094),
(41.84286,	-73.62103),
(41.76746,	-73.51102),
(41.75289,	-73.51743),
(41.75341,	-73.52047),
(41.73453,	-73.73219),
(41.7342,	-73.73128)
]

In [ ]:
# Initialize a dictionary to store the results
results = []


In [ ]:
# 1Function to find the nearest index for given coordinate in 1D arrays
#def find_nearest_index(lat_array, lon_array, lat_point, lon_point):
    #lat_idx = np.abs(lat_array - lat_point).argmin()
    #lon_idx = np.abs(lon_array - lon_point).argmin()
    #return lat_idx, lon_idx

In [ ]:
# 2Function to find the nearest point  
def find_nearest_index(lat_array, lon_array, lat_point, lon_point):  
    # Calculate squared distance from the provided coordinates to each grid point  
    dist_sq = (lat_array[:, np.newaxis] - lat_point)**2 + (lon_array[np.newaxis, :] - lon_point)**2  
    return np.unravel_index(np.argmin(dist_sq), dist_sq.shape)

In [ ]:
# Iterate over all NetCDF files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.nc'):
        file_path = os.path.join(folder_path, filename)

        # Extracting the date from the filename  
        date_str = filename.split('_')[4]  # Extract the date (e.g., "20190501T000000" becomes "20190501T")  
        date = date_str[:8]  # Get the date in YYYYMMDD format
        
        ds = xr.open_dataset(file_path)
        
        # Retrieve the latitude and longitude variables
        lat_candidates = ['latitude', 'lat']
        lon_candidates = ['longitude', 'lon']
        
        lat_var, lon_var = None, None
        for lat_name in lat_candidates:
            if lat_name in ds.variables:
                lat_var = ds[lat_name].values
                break

        for lon_name in lon_candidates:
            if lon_name in ds.variables:
                lon_var = ds[lon_name].values
                break

        if lat_var is None or lon_var is None:
            print(f"Skipping file {filename} due to missing latitude/longitude variables.")
            continue

        # Print lat/lon variables shape for debugging
        print(f"File: {filename}")
        print(f"  Lat shape: {lat_var.shape}, Lon shape: {lon_var.shape}")
        
        # Extract data for the coordinates
        for lat_point, lon_point in coordinates:
            try:
                lat_idx, lon_idx = find_nearest_index(lat_var, lon_var, lat_point, lon_point)
                print(f"  Closest index for latitude {lat_point}: {lat_idx}, longitude {lon_point}: {lon_idx}")
                print(f"  Closest lat value: {lat_var[lat_idx]}, lon value: {lon_var[lon_idx]}")
                
                row = {'File': filename, 'Date': date, 'Latitude': lat_point, 'Longitude': lon_point}
                
                for i in range(1, 15):
                    if 'BT_H' in ds.variables:
                        bt_h_data = ds['BT_H'].isel(lat=lat_idx, lon=lon_idx, inc=i-1).values
                        if np.isnan(bt_h_data):
                            print(f"    Extracted BT_H_inc{i}: NaN encountered, setting to None")
                            bt_h_data = None
                        else:
                            print(f"    Extracted BT_H_inc{i}: {bt_h_data}")
                        row[f'BT_H_inc{i}'] = bt_h_data
                    
                    if 'BT_V' in ds.variables:
                        bt_v_data = ds['BT_V'].isel(lat=lat_idx, lon=lon_idx, inc=i-1).values
                        if np.isnan(bt_v_data):
                            print(f"    Extracted BT_V_inc{i}: NaN encountered, setting to None")
                            bt_v_data = None
                        else:
                            print(f"    Extracted BT_V_inc{i}: {bt_v_data}")
                        row[f'BT_V_inc{i}'] = bt_v_data
                
                results.append(row)
                
            except KeyError as e:
                print(f"Skipping some variables in file {filename} due to missing key: {e}")
            except IndexError as e:
                print(f"Skipping file {filename} due to index error: {e}")
        ds.close()
        

In [ ]:
# Convert results to DataFrame
df = pd.DataFrame(results)

In [ ]:
df

In [ ]:
# Save DataFrame to Excel file
output_file = r'E:/SMOS20192022/TB_Result/SMOS_Temperature_Brightness_Result/Millbrook/Millbrook_TB_2022D.xlsx'
df.to_excel(output_file, index=False)

print(f"Data successfully saved to {output_file}")